In [2]:
from PIL import Image
from scipy.io import loadmat
import numpy as np
from pathlib import Path
from tqdm import tqdm
import itertools
from scipy.ndimage.morphology import distance_transform_edt
from collections import Counter
from itertools import chain
import json
import torch

from utils.show import show_layers_from_mask, show_layers_from_boundary

                        
def boundary_length_distribution(folder):
    lengths = []
    for file in folder:
        data = loadmat(file)
        for bscan in data["layerMaps"]:
            x_inds, _ = np.where(~np.isnan(bscan))
            if x_inds.any():
                lengths.append(x_inds[-1] - x_inds[0])
    return lengths


def boundary_middle_distribution(folder):
    middles = []
    for file in folder:
        data = loadmat(file)
        for bscan in data["layerMaps"]:
            x_inds, _ = np.where(~np.isnan(bscan))
            if x_inds.any():
                middles.append(x_inds[len(x_inds) // 2])
    return middles
            

                
def create_boundary_mask(boundary_array, height):
    mask = np.zeros((height, boundary_array.shape[1]), dtype="uint8")
    for col_idx, col in enumerate(boundary_array.T):
        if ~np.isnan(col).any():
            for boundary in col:
                mask[int(boundary), col_idx] = 1
    return mask


def create_layer_mask(boundary_array, height):
    mask = np.zeros((height, boundary_array.shape[1]), dtype="uint8")
    for col_idx, col in enumerate(boundary_array.T):
        prev_boundary = 0
        for boundary_idx, boundary in enumerate(col):
            mask[prev_boundary:int(boundary) + 1, col_idx] = boundary_idx
            prev_boundary = int(boundary) + 1
        mask[prev_boundary:, col_idx] = boundary_idx + 1
    return mask


def create_patches(img, lyr, patch_width, fluid):
    idx = np.where((np.isnan(lyr)).any(axis=0))[0]
    diff = np.diff(idx)
    useful_parts = diff >= patch_width
    useful_lengths = diff[useful_parts]
    useful_start_idx = idx[np.pad(useful_parts, (0, 1), constant_values=[False])]
    for ustart, ulength in zip(useful_start_idx, useful_lengths):
        number_of_shifts = (ulength - 1) // patch_width
        for shift_idx in range(number_of_shifts):
            indices = (slice(None), slice(
                ustart + 1 + shift_idx * patch_width, ustart + 1 + (shift_idx + 1) * patch_width
            ))
            img_patch = img[indices]
            lyr_patch = lyr[indices]
            fluid_patch = fluid[indices]
            if np.isnan(fluid_patch).any():
                fluid_patch = np.zeros_like(img_patch)
            else:
                fluid_patch = fluid_patch.astype(img_patch.dtype)
                fluid_patch[fluid_patch != 0] = 1
            mask = create_layer_mask(lyr_patch, img_patch.shape[0])
            yield img_patch, mask, fluid_patch, lyr_patch.T.tolist()
            
            
def generate_dme_dataset(input_dir, output_dir, patch_width):
    files = list(Path(input_dir).glob("*"))
    output_dir = Path(output_dir)
    output_dir.mkdir()
    cnt = 0
    boundary_indices_dict = {}
    layer_widths_dict = {}
    for file in tqdm(files, desc="data generation"):
        data = loadmat(file)
        layers = data["manualLayers1"].transpose((2, 0, 1))
        images = data["images"].transpose((2, 0, 1))
        fluids = data["manualFluid1"].transpose((2, 0, 1))
        for idx, (image, layer, fluid) in enumerate(zip(images, layers, fluids)):
            patch_generator = create_patches(image, layer, patch_width, fluid)
            for img, mask, fluid_patch, boundary_indices_list in patch_generator:
                Image.fromarray(img).save(output_dir / f"img_{cnt}.png")
                Image.fromarray(mask).save(output_dir / f"mask_{cnt}.png")
                Image.fromarray(fluid_patch).save(output_dir / f"fluid_{cnt}.png")
                boundary_indices_dict[cnt] = boundary_indices_list
                cnt += 1
    with open(output_dir / "boundary_indices.json", "w") as boundary_file:
        json.dump(boundary_indices_dict, boundary_file)

# LOAD FILES

In [129]:
amd = list(Path("../../dataset/raw/AMD/").glob("*"))
control = list(Path("../../dataset/raw/Control/").glob("*"))
# dme = list(Path("../../dataset/raw/2015_BOE_Chiu/").glob("*"))
dme = list(Path("../../dataset/raw/1-5").glob("*"))

# DME

In [130]:
dme

[PosixPath('../../dataset/raw/1-5/Subject_02.mat'),
 PosixPath('../../dataset/raw/1-5/Subject_04.mat'),
 PosixPath('../../dataset/raw/1-5/Subject_01.mat'),
 PosixPath('../../dataset/raw/1-5/Subject_03.mat'),
 PosixPath('../../dataset/raw/1-5/Subject_05.mat')]

In [131]:
data = loadmat(dme[0])
np.where((~np.isnan(data["manualLayers1"])).any(axis=(0, 1)))[0]

array([10, 15, 20, 25, 28, 30, 32, 35, 40, 45, 50])

In [132]:
np.where((~np.isnan(data["manualFluid1"])).any(axis=(0, 1)) & (data["manualFluid1"].sum((0, 1)) != 0))[0]

array([25, 28, 30, 32, 40, 45, 50])

In [134]:
# idx = 30
idx = 25
img = data["images"][..., idx]
lyr1 = data["manualLayers1"][..., idx]
lyr2 = data["manualLayers2"][..., idx]
fluid1 = data["manualFluid1"][..., idx]
fluid2 = data["manualFluid2"][..., idx]

# Image.fromarray(img).show()
# show_layers_from_boundary(img, lyr).show()
show_layers_from_boundary(img, lyr1, fluid=fluid1).show()
# show_layers_from_boundary(img, lyr2, fluid=fluid2).show()

In [38]:
l = np.where((~np.isnan(lyr1)).all(0))[0]

In [40]:
l.max() - l.min()

523

In [42]:
(np.diff(l) - 1).sum()

0

# Test

In [4]:
def show_layers_from_mask(img, mask, mean_std=None, normed=False):
    err_msg = "image and mask do not have the same dimensions"
    assert img.shape == mask.shape, err_msg
    if type(img) == torch.Tensor:
        img = img.cpu().numpy()
    if type(mask) == torch.Tensor:
        mask = mask.cpu().numpy()
    if mean_std:
        img = img * mean_std[1].numpy() + mean_std[0].numpy()
    if normed:
        img = np.asarray(img * 255, "uint8")
    dme_colorcode = {
        1: (170, 160, 250),
        2: (120, 200, 250),
        3: (80, 200, 250),
        4: (50, 230, 250),
        5: (20, 230, 250),
        6: (0, 230, 250),
        7: (0, 230, 100),
        9: (180, 255, 255)  # fluid
    }
    zeros = np.zeros_like(mask, dtype="uint8")
    hue = zeros.copy()
    saturation = zeros.copy()
    value = zeros.copy()
    alpha = zeros.copy()
    for klass, hsv in dme_colorcode.items():
        hue[mask == klass] = hsv[0]
        saturation[mask == klass] = hsv[1]
        value[mask == klass] = hsv[2]
        alpha[mask == klass] = 255
    img_stack = np.array([zeros, zeros, img]).transpose((1, 2, 0))
    img_img = Image.fromarray(img_stack, mode="HSV")
    colored_mask_stack = np.array([hue, saturation, value]).transpose((1, 2, 0))
    mask_img = Image.fromarray(colored_mask_stack, mode="HSV")
    alpha_img = Image.fromarray(alpha)
    img_w_layers = Image.composite(mask_img, img_img, alpha_img)
    return img_w_layers

In [6]:
# from utils.misc import get_loaders
# from utils.show import show_layers_from_mask
from utils.data import OCTDataset
from misc import get_mean_std
from torch.utils.data import Subset, DataLoader
import albumentations as A
from torch.nn import functional as F
from utils.misc import get_layer_channels

In [5]:
def get_loaders(
          data_dir,
          fluid,
          patch_width,
          batch_size,
          train_transform,
          num_workers,
):
    data_dir = Path(data_dir)
    train_ds = OCTDataset(data_dir / "training" / f"DME_{patch_width}", fluid)
    mean_std = get_mean_std(Subset(train_ds, range(len(train_ds))))
    norm = A.Normalize((mean_std[0],), (mean_std[1],), max_pixel_value=1., always_apply=True)
    if fluid:
        transf = A.Compose((norm, train_transform), additional_targets={"fluid": "mask"})
    else:
        transf = A.Compose((norm, train_transform))
    train_ds = OCTDataset(data_dir / "training" / f"DME_{patch_width}", fluid, transform=transf)
    valid_ds = OCTDataset(data_dir / "validation" / f"DME_{patch_width}", fluid, transform=norm)

    train_loader = DataLoader(
              train_ds,
              batch_size=batch_size,
              num_workers=num_workers,
              shuffle=False
    )
    valid_loader = DataLoader(
              valid_ds,
              batch_size=batch_size,
              num_workers=num_workers,
              shuffle=False
    )
    return train_loader, valid_loader, mean_std

In [172]:
train_dl, valid_dl, mean_std = get_loaders("../../generated", True, 64, 1, None, 2)
fluid_g = ((x, (mask, fluid)) for x, (mask, fluid) in train_dl if fluid.any())

In [242]:
def get_fluid_boundary(fluid):
    shifted = fluid[:, :, 1:] - fluid[:, :, :-1]
    x, y, z = torch.where(shifted == -1)
    z = z - 1
    shifted[shifted != 1] = 0
    shifted[x, y, z] = 1
    mask1 = torch.nn.functional.pad(shifted, (1, 0, 0, 0), "constant", 0)
    shifted = fluid[:, 1:, :] - fluid[:, :-1, :]
    x, y, z = torch.where(shifted == -1)
    y = y - 1
    shifted[shifted != 1] = 0
    shifted[x, y, z] = 1
    mask2 = torch.nn.functional.pad(shifted, (0, 0, 1, 0), "constant", 0)
    mask = torch.logical_or(mask1, mask2).int()
    return mask

In [253]:
x, (mask, fluid) = next(fluid_g)
show_layers_from_mask(x[0], mask[0], mean_std=mean_std, normed=True).show()
no_fluid = mask.clone()
no_fluid[fluid] = 9
show_layers_from_mask(x[0], no_fluid[0], mean_std=mean_std, normed=True).show()

boundary = F.pad(mask[:, 1:, :] - mask[:, :-1, :], (0, 0, 0, 1), "constant", 0)
# f = fluid
# shifted = f[:, :, 1:] - f[:, :, :-1]
# x, y, z = torch.where(shifted == -1)
# z = z - 1
# shifted[shifted != 1] = 0
# shifted[x, y, z] = 1
# mask1 = torch.nn.functional.pad(shifted, (1, 0, 0, 0), "constant", 0)
# shifted = f[:, 1:, :] - f[:, :-1, :]
# x, y, z = torch.where(shifted == -1)
# y = y - 1
# shifted[shifted != 1] = 0
# shifted[x, y, z] = 1
# mask2 = torch.nn.functional.pad(shifted, (0, 0, 1, 0), "constant", 0)

# mask = torch.logical_or(mask1, mask2)
mask = get_fluid_boundary(fluid)
bf = torch.logical_or(boundary, mask)

# Image.fromarray(boundary.squeeze().numpy() * 255).show()
Image.fromarray(bf.byte().squeeze().numpy() * 255).show()
# Image.fromarray(mask.byte().squeeze().numpy() * 255).show()

/home/loerinczy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  ../aten/src/ATen/native/TensorAdvancedIndexing.cpp:1273.)
  after removing the cwd from sys.path.


In [283]:
import torch
from utils.misc import dice_coefficient
from torch.nn import functional as F
from torch import nn


class DiceLoss(nn.Module):

    def __init__(self, weight_channel=None, num_classes=9):
        super(DiceLoss, self).__init__()
        self.weight_channel = (
            weight_channel if weight_channel
            else torch.zeros(num_classes)
        )
        self.num_classes = num_classes

    def forward(self, predictions: torch.Tensor, targets: torch.Tensor):
        """
        Computes the dice loss.
        :param predictions: torch.Tensor of shape N x C x H x W,
            the raw prediction of the model
        :param targets: torch.Tensor of shape N x H x W,
            the raw target from the dataset
        :param weights: torch.Tensor of shape C,
            per channel weights for loss weighting
        :return: the dice loss summed over the channels, averaged over the batch
        """
        predictions_exp = predictions.exp()
        dice_coeff = dice_coefficient(predictions_exp, targets, self.num_classes).mean(0)
        weight = 1 + self.weight_channel
        weight /= weight.sum()
        loss = (weight * (1 - dice_coeff)).sum()
        return loss


class CombinedLoss(nn.Module):

    def __init__(
              self,
              num_classes=9,
              weight_channel_cross=None,
              weight_channel_dice=None,
              weight_boundary_cross=1.
    ):
        super(CombinedLoss, self).__init__()
        self.cross_entropy_loss_fn = nn.CrossEntropyLoss()
        self.dice_loss_fn = DiceLoss(weight_channel_dice, num_classes)
        self.weight_channel_cross = (
            weight_channel_cross if weight_channel_cross 
            else torch.ones(num_classes)
        )
        self.weight_boundary_cross = weight_boundary_cross
        self.num_classes = num_classes

    def forward(self, predictions, targets):
        if self.num_classes == 10:
            targets, fluid = targets
            fluid_boundary = get_fluid_boundary(fluid)
        weight_boundary = F.pad(targets[:, 1:, :] - targets[:, :-1, :], (0, 0, 0, 1), "constant", 0)
        if self.num_classes == 10:
            weight_boundary += fluid_boundary
            targets[fluid.bool()] = 9
        weight_boundary = weight_boundary * self.weight_boundary_cross
        weight_channel = torch.zeros(*targets.shape, self.num_classes, device=targets.device).scatter_(
                  -1, targets.unsqueeze(-1), 1
        )
        weight_channel = weight_channel * self.weight_channel_cross
        weight_channel = weight_channel.max(-1).values
        weight = 1 + weight_boundary + weight_channel
        weight /= weight.sum()
        cross_entropy_loss = (self.cross_entropy_loss_fn(predictions, targets) * weight).sum()
        dice_loss = self.dice_loss_fn(predictions, targets)
        return cross_entropy_loss, dice_loss


In [295]:
loss = CombinedLoss(num_classes=9)

In [296]:
# x, (mask, fluid) = next(fluid_g)

In [297]:
pred = get_layer_channels(mask.long(), 9)

In [298]:
# loss(pred, (mask.long(), fluid))
loss(pred, mask.long())

(tensor(1.3720), tensor(0.9547))

In [299]:
import timeit

3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


KeyboardInterrupt: 

In [293]:
mask = get_fluid_boundary(fluid)

## Add fluid boundary weighting

In [242]:
Image.fromarray(255 * fluid1).show()
# Image.fromarray(254 * shifted).show()
# Image.fromarray(254 * mask).show()

fluid = fluid1

shifted = fluid[:, 1:] - fluid[:, :-1]
x, y = np.where(shifted == -1)
y = y - 1
shifted[shifted != 1] = 0
shifted[x, y] = -1
shifted[shifted != 0] = 1
mask1 = np.pad(shifted, ((0, 0), (1, 0)), "constant", constant_values=0)
shifted = fluid[1:, :] - fluid[:-1, :]
x, y = np.where(shifted == -1)
x = x - 1
shifted[shifted != 1] = 0
shifted[x, y] = -1
shifted[shifted != 0] = 1
mask2 = np.pad(shifted, ((1, 0), (0, 0)), "constant", constant_values=0)

mask = np.logical_or(mask1, mask2).astype(float)

img2 = Image.fromarray(
    np.array([
        np.ones_like(fluid, dtype="uint8") * 230,
        np.ones_like(fluid, dtype="uint8") * 190,
        np.ones_like(fluid, dtype="uint8") * 255]).transpose((1, 2, 0)), mode="HSV")

img1 = Image.fromarray(np.array([ 
    np.zeros_like(fluid).astype("uint8"),
    np.zeros_like(fluid).astype("uint8"),
    fluid.astype("uint8") * 255,]).transpose((1, 2, 0)), mode="HSV")

maskimg = Image.fromarray(np.array([
    mask.astype("uint8"),
    np.zeros_like(mask).astype("uint8"),
    np.zeros_like(mask).astype("uint8")]).transpose((1, 2, 0)))

Image.composite(img2, img1, Image.fromarray(mask.astype("uint8") * 255)).show()

np.logical_and(fluid1, mask).sum() == mask.sum()

Image.fromarray(np.logical_and(mask, fluid)).show()
Image.fromarray(np.logical_and(mask, ~np.logical_and(mask, fluid))).show()

### Shift torch version

In [243]:
shifted = f[:, 1:] - f[:, :-1]
x, y = torch.where(shifted == -1)
y = y - 1
shifted[shifted != 1] = 0
shifted[x, y] = 1
mask1 = torch.nn.functional.pad(shifted, (1, 0, 0, 0), "constant", 0)
shifted = f[1:, :] - f[:-1, :]
x, y = torch.where(shifted == -1)
x = x - 1
shifted[shifted != 1] = 0
shifted[x, y] = 1
mask2 = torch.nn.functional.pad(shifted, (0, 0, 1, 0), "constant", 0)
mask = torch.logical_or(mask1, mask2).int()

In [7]:
idx = 12
img = np.array(Image.open(f"../../generated/DME_496/img_{idx}.png"))
lyr = np.array(Image.open(f"../../generated/DME_496/mask_{idx}.png"))

In [8]:
show_layers_from_mask_array(img, lyr).show()

In [13]:
show_layers_from_boundary_array(img, lyr2.T).show()
show_layers_from_boundary_array(img, lyr.T).show()

In [320]:
data = loadmat("../../dataset/raw/2015_BOE_Chiu/Subject_02.mat")

In [321]:
idx = 28

In [322]:
img = data["images"][..., idx]
lyr = data["manualLayers1"][..., idx]

In [323]:
show_layers_from_boundary_array(img, lyr.T).show()

In [65]:
segmented_length = list()

for d in range(10):
    data = loadmat(dme[d])
    for lyr_ind in range(61):
        m1 = data["manualLayers1"][..., lyr_ind]
        m2 = data["manualLayers2"][..., lyr_ind]
        lengths = [] 
        if (~np.isnan(m1)).any():
            w_idx = np.where(~np.isnan(m1))[1]
            delta = w_idx[-1] - w_idx[0]
            lengths.append(delta)
        else:
            lengths.append(0)
        if (~np.isnan(m2)).any():
            w_idx = np.where(~np.isnan(m2))[1]
            delta = w_idx[-1] - w_idx[0]
            lengths.append(delta)
        else:
            lengths.append(0)
        if sum(lengths):
            segmented_length.append(tuple(lengths))
            
Counter(segmented_length)

Counter({(523, 523): 33,
         (547, 547): 11,
         (535, 535): 11,
         (541, 541): 33,
         (505, 505): 11,
         (499, 499): 11})

In [5]:
dme496img = sorted(list(Path("../../generated/DME_496/").glob("img*")))
dme496mask = sorted(list(Path("../../generated/DME_496").glob("mask*")))

In [6]:
img = np.array(Image.open(dme496img[0]))
mask = np.array(Image.open(dme496mask[0]))

In [8]:
show_layers_from_mask_array(img, mask).show()

# There are parts where not all the layers are annotated

In [6]:
# for d in range(10):
#     data = loadmat(dme[d])
#     lyr = data["manualLayers1"]
#     if (np.isnan(lyr).any(axis=0) != np.isnan(lyr).all(axis=0)).sum():
#         print(d)

data = loadmat(dme[0])
lyr = data["manualLayers1"]
img = data["images"]
idx = (np.isnan(lyr).any(axis=0) != np.isnan(lyr).all(axis=0))
np.isnan(lyr[:, idx]).sum(axis=-1)

array([  0,   0,   0,   0, 127,   0,   0,   0])

# Generating the dataset

In [39]:
patch_width = 64
generate_dme_dataset("../../dataset/raw/1-5", f"../../generated/training/DME_{patch_width}", patch_width)
generate_dme_dataset("../../dataset/raw/6-10", f"../../generated/validation/DME_{patch_width}", patch_width)
len(list(Path(f"../../generated/training/DME_{patch_width}").glob("img*")))

data generation: 100%|██████████| 5/5 [00:09<00:00,  1.90s/it]


335

# Proove that the layer mask generation is correct

In [149]:
bmask = create_boundary_mask(lyr[:, 500:600], 496)
lmask = create_layer_mask(lyr[:, 500:600], 496)
diff = np.diff(lmask, axis=0)
(np.pad(diff, ((0, 1), (0, 0)), constant_values=(0,)) != bmask).sum()

0

# AMD

In [10]:
data = loadmat(amd[0])
idx = 30
img = data["images"][:, :, idx]
lyr = data["layerMaps"][idx]

In [14]:
def generate_amd_dataset(amd_dir, control_dir, output_dir, patch_width):
    files = list(
        chain(
            Path(amd_dir).glob("*"),
            Path(control_dir).glob("*")
        )
    )
    output_dir = Path(output_dir)
    output_dir.mkdir()
    cnt = 0
    for file in tqdm(files, desc="data generation"):
        data = loadmat(file)
        layers = data["layerMaps"].transpose((0, 2, 1))
        images = data["images"].transpose((2, 0, 1))
        for idx, (image, layer) in enumerate(zip(images, layers)):
            patch_generator = create_patches(image, layer, patch_width)
            for img, mask in patch_generator:
                Image.fromarray(img).save(output_dir / f"img_{cnt}.png")
                Image.fromarray(mask).save(output_dir / f"mask_{cnt}.png")
                cnt += 1

In [34]:
patch_width = 736
generate_amd_dataset(
    "../../dataset/raw/AMD/",
    "../../dataset/raw/Control/",
    f"../../generated/AMD_{patch_width}",
    patch_width
)

data generation: 100%|██████████| 384/384 [04:31<00:00,  1.42it/s]


In [35]:
len(list(Path("../../generated/AMD_736").glob("img*")))

2901

In [26]:
loadmat(d)

784

In [33]:
736 / 16

46.0

In [11]:
show_layers_from_boundary_array(img, lyr).show()
show_boundary_from_boundary_array(img, lyr).show()